## Algoritmos de decodificación para códigos BCH
#### Autor: Javier Ojeda Baena
#### DNI: 75943022N

In [1]:
from sage.coding.bch_code import BCHCode
from sage.coding.cyclic_code import CyclicCode
from sage.coding.encoder import Encoder
from sage.coding.decoder import Decoder
from IPython.display import HTML

In [2]:
def random_word(n, F):
    """
    Genera una palabra aleatoria de longitud n utilizando los elementos de un cuerpo finito F.

    Parámetros:
        - n : longitud de la palabra que se quiere generar.
        - F : cuerpo finito del que se van a coger los elementos para generar la palabra aleatoria.

    Devuelve:
        Palabra aleatoria de longitud n con elementos de F
    """
    word = []

    while len(word) != n:
        word = word + [choice(F.list())]

    word = vector(word, F)
    
    return word

In [3]:
def random_error(n, n_errors, F):
    """
    Genera un vector de errores aleatorios de longitud n, que tiene como máximo n errors 
    elementos no nulos, utilizando elementos del cuerpo finito F.

    Parámetros:
        - n : longitud del vector de errores que se quiere generar.
        - n_errors : número máximo de elementos no nulos que se quieren en el vector de errores.
        - F : cuerpo finito del que se van a coger los elementos para generar el vector de errores aleatorios.
        
    Devuelve:
        - Vector de errores aleatorios con n elementos, n_errors elementos no nulos con elementos de F
    """
    e = [0] * n
    
    i = 0
    
    while i < n_errors:
        p = randint(0, n-1)
        e[p] = choice(F.list())
        i += 1

    e = vector(e, F)
    
    return e

In [4]:
class BCHEncoder(Encoder):
    """
    Clase que representa un codificador BCH.
    """
    
    
    def __init__(self, code):
        """
        Constructor de la clase.

        Parámetros:
            - code: código BCH.
        """
        super(BCHEncoder, self).__init__(code)
        
        
    def _repr_(self):
        """
        Método que devuelve una representación en cadena de caracteres de la clase.

        Devuelve:
            - Cadena de caracteres que representa el objeto.
        """
        return "Encoder for {}".format(self.code())
    
    
    def encode (self, m):
        """
        Método que codifica un mensaje utilizando el código BCH.

        Parámetros:
            - m: mensaje a codificar.

        Devuelve:
            - Mensaje codificado con el código BCH.
        """
        return m * self.code().generator_matrix()

## Decodificador con el algoritmos de Peterson-Gorenstein-Zieler
1. Calcular los sindrome $S_{i}=y(\xi^{i})$ para $1\leq i	\leq 2t$.


2. Sea:

$$M_{\mu}=\begin{pmatrix}
S_{1} & S_{2} & \cdots &  S_{\mu}\\
S_{2} & S_{3} & \cdots &  S_{\mu+1}\\
 & & \vdots & \\
S_{\mu} & S_{\mu+1} & \cdots & S_{2\mu-1}\\
\end{pmatrix}$$

En el orden $\mu=t,\,\mu=t-1,\,\ldots$ decidimos si $M_{\mu}$ es singular, parando al primer valor de $\mu$ para el que $M_{\mu}$ es no singular. Tomamos $v=\mu$ y resolvermos el siguiente sistema para determinar $\sigma(x)$:

$$\begin{pmatrix}
S_{1} & S_{2} & S_{3} & \cdots & S_{v-1} & S_{v}\\
S_{2} & S_{3} & S_{4} & \cdots & S_{v} & S_{v+1}\\
S_{3} & S_{4} & S_{5} & \cdots & S_{v+1} & S_{v+2}\\
 & & & \vdots & & \\
S_{v} & S_{v+1} & S_{v+2} & \cdots & S_{2v-2} & S_{2v-1}\\
\end{pmatrix}\begin{pmatrix}
\sigma_{v}\\
\sigma_{v-1}\\
\sigma_{v-2}\\
\vdots\\
\sigma_{1}
\end{pmatrix}=\begin{pmatrix}
-S_{v+1}\\
-S_{v+2}\\
-S_{v+3}\\
\vdots\\
-S_{2v}
\end{pmatrix}$$


3. Entontramos las raices de $\sigma(x)$ calculando $\sigma(\xi^{i})$ para $0\leq i<n$. Invertimos las raices para obtener las posiciones de los errores $X_{j}$.


4. Resolvemos las primeras $v$ ecuaciones del siguiente sistema para obterner las magnitudes de los errores $E_{j}$:

$$S_{1}=E_{1}X_{1}+E_{2}X_{2}+\cdots+E_{v}X_{v}$$
$$S_{2}=E_{1}X_{1}^{2}+E_{2}X_{2}^{2}+\cdots+E_{v}X_{v}^{2}$$
$$S_{3}=E_{1}X_{1}^{3}+E_{2}X_{2}^{3}+\cdots+E_{v}X_{v}^{3}$$
$$\vdots$$
$$S_{2t}=E_{1}X_{1}^{2t}+E_{2}X_{2}^{2t}+\cdots+E_{v}X_{v}^{2t}$$

In [5]:
class BCHDecoderPGZ(Decoder):
    """
    Implementación del decodificador con el algoritmo de Peterson-Gorenstein-Zieler
    para códigos BCH.
    """
        
    
    def __init__(self, code):
        """
        Constructor de la clase.

        Parámetros:
            - code: código BCH.
        """
        super(BCHDecoderPGZ, self).__init__(code, code.ambient_space(), "BCHDecoderPGZ")
    
    
    def _repr_(self):
        """
        Método que devuelve una representación en cadena de caracteres de la clase.

        Devuelve:
            - cadena de caracteres que representa el objeto.
        """
        return "Peterson-Gorenstein-Zieler decoder for {}".format(self.code())  

    
    
    def decode_to_code(self, y):
        """
        Dado un vector y calcula la palabra código más cercana a este con el algoritmo de
        Peterson-Gorenstein-Zieler

        Parámetros:
            - y: vector a decodificar.

        Devuelve:
            - La palabra código más cercana a y.
        """
        
        F = self.code().base_field()
        q = len(F.list())
        n = self.code().length()
        
        m = 1
        
        while (q**m%n != 1):
            m += 1
            
        L = GF(q^m)
        R.<x> = L[]    

        # Paso 1: calculamos S e inicializamos los primeros valores del algoritmo
        
        t = floor((self.code().designed_distance()-1)/2)
        primitive_root = GF(q^m).gen()**((q**m-1)//n)
        
        y_polynomial = 0
        for i in range(len(y)):
            y_polynomial += y[i]*x**i

        S = vector([y_polynomial(primitive_root**i) for i in range(1, 2*t+1)])
        
        if S.is_zero():
            return y

        mu = t
        
        M = []
        
        for i in range(mu):
            M.append(S[i:(i+mu)])
            
        M = matrix(M)
        
        while M.is_singular():
            mu -= 1
            
            M = []
        
            for i in range(mu):
                M.append(S[i:(i+mu)])

            M = matrix(M)
        
        v = mu
        
        A = []
        
        for i in range(v):
            A.append(S[i:(i+v)])
            
        A = matrix(A)
        
        s = A.solve_right(-S[v:(2*v)])
        
        sigma = 1
        
        for i in range(v):
            sigma = sigma + s[-i-1]*x**(i+1)
        
        # Paso 3: calculamos las posiciones de los errores, sabiendo que son los inversos de las raices
        #         de sigma(x)
        
        xi = [primitive_root**i for i in range(n)]
        
        k = []
        for root in sigma.roots():
            k.append(xi.index(root[0]**(-1)))
        
        # Paso 4: resolvemos el sistema formado por los sindromes, las posiciones de los errores, y las
        #         magnitudes de los mismos
        
        A = []
        
        for i in range(1, 2*t+1):
            equation = []
            for root in sigma.roots():
                equation.append(root[0]**(-i))
            A.append(equation)
            
        A = matrix(A)
        
        E = A.solve_right(S)
        
        # Calculamos el vector de errores
        
        e = [0] * n
        
        v = len(k)
        
        for i in range(v):
            e[k[i]] = E[i]
            
        y_decoded = y-vector(e, F)
        
        return y_decoded

    def decode_to_message(self, y):
        """
        Dado un vector y lo decodifica con el algoritmo de Peterson-Gorenstein-Zieler obteniendo
        el mensaje original.

        Parámetros:
            - y: vector a decodificar.

        Devuelve:
            - Mensaje decodificado.
        """
        c_decoded = self.decode_to_code(y)
        word = self.code().generator_matrix().solve_left(c_decoded)
        return word

## Decodificador con el algoritmos de Sugiyama
El proceso final coincide con el algoritmo de Peterson-Gorenstein-Ziele, la única parte que cambia es la forma de calcular $\sigma(x)$ el polinomio localizador de errores:

1. Calculamos el síndrome $S(x)=\displaystyle\sum_{i=0}^{2t-1} S_{i+1}x^{i}$.


2. Sean $r_{-1}(x) = x^{2t}$, $r_0(x) = S(x)$, $b_{-1}(x) = 0$ y $b_0(x) = 1$. Repetimos los siguientes dos calculos encontrando $h_{i}(x),\,r_{i}(x),\,\text{ y }b_{i}(x)$ de forma inductiva para $i=1,\,2,\,\ldots,\,I$, hasta $I$ satisfaciendo $grad(r_{I-1}(x)\geq t$ y $grad(r_{I}(x))<t$:
$$r_{i-2}(x)=r_{i-1}(x)h_{i}+r_{i}(x),\text{ donde }grad(r_{i}(x))<grad(r_{i-1}(x))$$
$$b_{i}(x)=b_{i-2}(x)-h_{i}(x)b_{i-1}(x)$$


3. $\sigma(x)$ es algun escalar distinto de cero multiplicado por $b_{I}(x)$.

In [6]:
class BCHDecoderS(Decoder):
    """
    Implementación del decodificador con el algoritmo de Sugiyama para códigos BCH.
    """
        
    
    def __init__(self, code):
        """
        Constructor de la clase.

        Parámetros:
            - code: código BCH.
        """
        super(BCHDecoderS, self).__init__(code, code.ambient_space(), "BCHDecoderS")
    
    
    def _repr_(self):
        """
        Método que devuelve una representación en cadena de caracteres de la clase.

        Devuelve:
            - Cadena de caracteres que representa el objeto.
        """
        return "Sugiyama decoder for {}".format(self.code())  

    
    
    def decode_to_code(self, y):
        """
        Dado un vector y calcula la palabra código más cercana a este con el algoritmo de Sugiyama

        Parámetros:
            - y: vector a decodificar.

        Devuelve:
            - La palabra código más cercana a y.
        """
        
        F = self.code().base_field()
        q = len(F.list())
        n = self.code().length()
        
        m = 1
        
        while (q**m%n != 1):
            m += 1
            
        L = GF(q^m)
        R.<x> = L[] 

        # Paso 1: calculamos S(x) e inicializamos los primeros valores del algoritmo
        
        t = floor((self.code().designed_distance()-1)/2)
        primitive_root = GF(q^m).gen()**((q**m-1)//n)
        
        y_polynomial = 0
        for i in range(len(y)):
            y_polynomial += y[i]*x**i

            
        S = vector([y_polynomial(primitive_root**i) for i in range(1, 2*t+1)])
        
        S_polynomial = 0

        for j in range(len(S)):
            S_polynomial = S_polynomial+S[j]*x**j

        if S_polynomial == 0:
            return y
        
        i = 1
        
        f = x**(2*t)
        s = S_polynomial
        r = [f, s]
        b = [0, 1]
        
        # Paso 2: calculamos el el polinomio bI(x)

        while r[i].degree() >= t or r[i-1].degree() < t:
            (q, rs) = r[i-1].quo_rem(r[i])
            r.append(rs)
            b.append(b[i-1]-q*b[i])
            i += 1
        
        
        # Paso 3: el polinomio localizador de errores, sigma, es bI(x) por algún escalar distinto de 0
        
        sigma = b[i] / b[i].coefficients()[-1]
        
        # Paso 4: calculamos las posiciones de los errores, sabiendo que son los inversos de las raices
        #         de sigma(x)
        
        xi = [primitive_root**i for i in range(n)]
        
        k = []
        for root in sigma.roots():
            k.append(xi.index(root[0]**(-1)))
        
        # Paso 5: resolvemos el sistema formado por los sindromes, las posiciones de los errores, y las
        #         magnitudes de los mismos
        
        A = []
        
        for i in range(1, 2*t+1):
            equation = []
            for root in sigma.roots():
                equation.append(root[0]**(-i))
            A.append(equation)
            
        A = matrix(A)
        
        E = A.solve_right(S)
        
        # Calculamos el vector de errores
        
        e = [0] * n
        
        v = len(k)
        
        for i in range(v):
            e[k[i]] = E[i]
            
        y_decoded = y-vector(e, F)
        
        return y_decoded

    def decode_to_message(self, y):
        """
        Dado un vector y lo decodifica con el algoritmo de Sugiyama obteniendo el mensaje original.

        Parámetros:
            - y: vector a decodificar.

        Devuelve:
            - El mensaje decodificado.
        """
        c_decoded = self.decode_to_code(y)
        word = self.code().generator_matrix().solve_left(c_decoded)
        return word

## Ejemplo 1

### Generación del código y la palabra a codificar

Creamos un código BCH sobre $\mathbb{F}_{5}$, de longitud $11$ y distancia diseñada $2$:

In [7]:
q = 5
F = GF(q)
n = 11
delta = 2
C = BCHCode(F, n, delta, None)
d = C.generator_matrix().nrows()

print("Código BCH:")
print(C)

print("\nMatriz de paridad:")
show(C.parity_check_matrix())

print("\nMatriz generadora:")
show(C.generator_matrix())

print("\nPolinomio generador:")
show(CyclicCode(code = C).generator_polynomial())

Código BCH:
[11, 6] BCH Code over GF(5) with designed distance 2

Matriz de paridad:


/usr/lib/python3/dist-packages/sage/coding/cyclic_code.py:451: FutureWarning: This class/method/function is marked as experimental. It, its functionality or its interface might change without a formal deprecation.
See http://trac.sagemath.org/20284 for details.
  FE = RelativeFiniteFieldExtension(Fsplit, F,


[1 1 2 2 0 3 1 0 0 0 0]
[0 1 1 2 2 0 3 1 0 0 0]
[0 0 1 1 2 2 0 3 1 0 0]
[0 0 0 1 1 2 2 0 3 1 0]
[0 0 0 0 1 1 2 2 0 3 1]


Matriz generadora:


[4 3 1 4 4 1 0 0 0 0 0]
[0 4 3 1 4 4 1 0 0 0 0]
[0 0 4 3 1 4 4 1 0 0 0]
[0 0 0 4 3 1 4 4 1 0 0]
[0 0 0 0 4 3 1 4 4 1 0]
[0 0 0 0 0 4 3 1 4 4 1]


Polinomio generador:


x^5 + 4*x^4 + 4*x^3 + x^2 + 3*x + 4

Generamos una palabra aleatoria de tamaño $n$ de elementos del cuerpo finito, $\mathbb{F}_{5}$.

In [8]:
Encoder = BCHEncoder(C)

print("Codificador BCH:")
print(Encoder)

print("\nPalabra para codificar:")
message = random_word(d, F)
show(message)

Codificador BCH:
Encoder for [11, 6] BCH Code over GF(5) with designed distance 2

Palabra para codificar:


(0, 2, 4, 0, 1, 4)

Codificamos la palabra.

In [9]:
print("\nPalabra codificada:")
c = Encoder.encode(message)
show(c)


Palabra codificada:


(0, 3, 2, 4, 1, 3, 1, 2, 0, 2, 4)

Generamos un vector de errores y se lo agregamos a la palabra codidicada.

In [10]:
print("\nVector de errores:")
n_errors = floor((delta-1)/2)
e = random_error(len(c), n_errors, F)
show(e)

print("\nPalabra codificada con errores (y = c + e):")
y = c + e
show(y)


Vector de errores:


(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)


Palabra codificada con errores (y = c + e):


(0, 3, 2, 4, 1, 3, 1, 2, 0, 2, 4)

### Algoritmo de Peterson-Gorenstein-Zieler
Decodificamos con el algoritmo de Peterson-Gorenstein Zieler y comprobamos si coincide con el mensaje original.

In [11]:
Decoder = BCHDecoderPGZ(C)

print("\nDecodificador BCH con el algoritmo de Peterson-Gorenstein-Zieler:")
print(Decoder)

print("\nDecodificacion de y:")
c_decoded = Decoder.decode_to_code(y)
show(c_decoded)

print("\nMensaje decodificado de y:")
message_decoded = Decoder.decode_to_message(y)
show(message_decoded)


Decodificador BCH con el algoritmo de Peterson-Gorenstein-Zieler:
Peterson-Gorenstein-Zieler decoder for [11, 6] BCH Code over GF(5) with designed distance 2

Decodificacion de y:


(0, 3, 2, 4, 1, 3, 1, 2, 0, 2, 4)


Mensaje decodificado de y:


(0, 2, 4, 0, 1, 4)

In [12]:
if c == c_decoded:
    mensaje = "<p style='color:green; font-size:16px;'></br>El resultado de la decodificación coincide con la codificación de la palabra original</p>"
else:
    mensaje = "<p style='color:red; font-size:16px;'></br>El resultado de la decodificación coincide no con la codificación de la palabra original</p>"

HTML(mensaje)

In [13]:
if message == message_decoded:
    mensaje = "<p style='color:green; font-size:16px;'></br>El mensaje decodificado coincide con el original</p>"
else:
    mensaje = "<p style='color:red; font-size:16px;'></br>El mensaje decodificado no coincide con el original</p>"

HTML(mensaje)

### Algoritmo de Sugiyama
Decodificamos con el algoritmo de Sugiyama y comprobamos si coincide con el mensaje original.

In [14]:
Decoder = BCHDecoderS(C)

print("\nDecodificador BCH con el algoritmo de Sugiyama:")
print(Decoder)

print("\nDecodificacion de y:")
c_decoded = Decoder.decode_to_code(y)
show(c_decoded)

print("\nMensaje decodificado de y:")
message_decoded = Decoder.decode_to_message(y)
show(message_decoded)


Decodificador BCH con el algoritmo de Sugiyama:
Sugiyama decoder for [11, 6] BCH Code over GF(5) with designed distance 2

Decodificacion de y:


(0, 3, 2, 4, 1, 3, 1, 2, 0, 2, 4)


Mensaje decodificado de y:


(0, 2, 4, 0, 1, 4)

In [15]:
if c == c_decoded:
    mensaje = "<p style='color:green; font-size:16px;'></br>El resultado de la decodificación coincide con la codificación de la palabra original</p>"
else:
    mensaje = "<p style='color:red; font-size:16px;'></br>El resultado de la decodificación coincide no con la codificación de la palabra original</p>"

HTML(mensaje)

In [16]:
if message == message_decoded:
    mensaje = "<p style='color:green; font-size:16px;'></br>El mensaje decodificado coincide con el original</p>"
else:
    mensaje = "<p style='color:red; font-size:16px;'></br>El mensaje decodificado no coincide con el original</p>"

HTML(mensaje)

## Ejemplo 2

### Generación del código y la palabra a codificar

Creamos un código BCH sobre $\mathbb{F}_{3^2}$, de longitud $13$ y distancia diseñada $5$:

In [17]:
q = 3^2
F = GF(q)
n = 13
delta = 5
C = BCHCode(F, n, delta, None)
d = C.generator_matrix().nrows()

print("Código BCH:")
print(C)

print("\nMatriz de paridad:")
show(C.parity_check_matrix())

print("\nMatriz generadora:")
show(C.generator_matrix())

print("\nPolinomio generador:")
show(CyclicCode(code = C).generator_polynomial())

Código BCH:
[13, 4] BCH Code over GF(9) with designed distance 5

Matriz de paridad:


[1 2 2 0 1 0 0 0 0 0 0 0 0]
[0 1 2 2 0 1 0 0 0 0 0 0 0]
[0 0 1 2 2 0 1 0 0 0 0 0 0]
[0 0 0 1 2 2 0 1 0 0 0 0 0]
[0 0 0 0 1 2 2 0 1 0 0 0 0]
[0 0 0 0 0 1 2 2 0 1 0 0 0]
[0 0 0 0 0 0 1 2 2 0 1 0 0]
[0 0 0 0 0 0 0 1 2 2 0 1 0]
[0 0 0 0 0 0 0 0 1 2 2 0 1]


Matriz generadora:


[2 0 2 2 0 1 0 2 1 1 0 0 0]
[0 2 0 2 2 0 1 0 2 1 1 0 0]
[0 0 2 0 2 2 0 1 0 2 1 1 0]
[0 0 0 2 0 2 2 0 1 0 2 1 1]


Polinomio generador:


x^9 + x^8 + 2*x^7 + x^5 + 2*x^3 + 2*x^2 + 2

Generamos una palabra aleatoria de tamaño $n$ de elementos del cuerpo finito, $\mathbb{F}_{3^2}$.

In [18]:
Encoder = BCHEncoder(C)

print("Codificador BCH:")
print(Encoder)

print("\nPalabra para codificar:")
message = random_word(d, F)
show(message)

Codificador BCH:
Encoder for [13, 4] BCH Code over GF(9) with designed distance 5

Palabra para codificar:


(z2 + 2, z2 + 1, 1, z2 + 2)

Codificamos la palabra.

In [19]:
print("\nPalabra codificada:")
c = Encoder.encode(message)
show(c)


Palabra codificada:


(2*z2 + 1, 2*z2 + 2, 2*z2, 1, 2*z2 + 1, 2, 2, 2*z2 + 2, z2, 2*z2 + 2, 0, z2, z2 + 2)

Generamos un vector de errores y se lo agregamos a la palabra codidicada.

In [20]:
print("\nVector de errores:")
n_errors = floor((delta-1)/2)
e = random_error(len(c), n_errors, F)
show(e)

print("\nPalabra codificada con errores (y = c + e):")
y = c + e
show(y)


Vector de errores:


(1, 2*z2 + 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)


Palabra codificada con errores (y = c + e):


(2*z2 + 2, z2 + 1, 2*z2, 1, 2*z2 + 1, 2, 2, 2*z2 + 2, z2, 2*z2 + 2, 0, z2, z2 + 2)

### Algoritmo de Peterson-Gorenstein-Zieler
Decodificamos con el algoritmo de Peterson-Gorenstein Zieler y comprobamos si coincide con el mensaje original.

In [21]:
Decoder = BCHDecoderPGZ(C)

print("\nDecodificador BCH con el algoritmo de Peterson-Gorenstein-Zieler:")
print(Decoder)

print("\nDecodificacion de y:")
c_decoded = Decoder.decode_to_code(y)
show(c_decoded)

print("\nMensaje decodificado de y:")
message_decoded = Decoder.decode_to_message(y)
show(message_decoded)


Decodificador BCH con el algoritmo de Peterson-Gorenstein-Zieler:
Peterson-Gorenstein-Zieler decoder for [13, 4] BCH Code over GF(9) with designed distance 5

Decodificacion de y:


(2*z2 + 1, 2*z2 + 2, 2*z2, 1, 2*z2 + 1, 2, 2, 2*z2 + 2, z2, 2*z2 + 2, 0, z2, z2 + 2)


Mensaje decodificado de y:


(z2 + 2, z2 + 1, 1, z2 + 2)

In [22]:
if c == c_decoded:
    mensaje = "<p style='color:green; font-size:16px;'></br>El resultado de la decodificación coincide con la codificación de la palabra original</p>"
else:
    mensaje = "<p style='color:red; font-size:16px;'></br>El resultado de la decodificación coincide no con la codificación de la palabra original</p>"

HTML(mensaje)

In [23]:
if message == message_decoded:
    mensaje = "<p style='color:green; font-size:16px;'></br>El mensaje decodificado coincide con el original</p>"
else:
    mensaje = "<p style='color:red; font-size:16px;'></br>El mensaje decodificado no coincide con el original</p>"

HTML(mensaje)

### Algoritmo de Sugiyama
Decodificamos con el algoritmo de Sugiyama y comprobamos si coincide con el mensaje original.

In [24]:
Decoder = BCHDecoderS(C)

print("\nDecodificador BCH con el algoritmo de Sugiyama:")
print(Decoder)

print("\nDecodificacion de y:")
c_decoded = Decoder.decode_to_code(y)
show(c_decoded)

print("\nMensaje decodificado de y:")
message_decoded = Decoder.decode_to_message(y)
show(message_decoded)


Decodificador BCH con el algoritmo de Sugiyama:
Sugiyama decoder for [13, 4] BCH Code over GF(9) with designed distance 5

Decodificacion de y:


(2*z2 + 1, 2*z2 + 2, 2*z2, 1, 2*z2 + 1, 2, 2, 2*z2 + 2, z2, 2*z2 + 2, 0, z2, z2 + 2)


Mensaje decodificado de y:


(z2 + 2, z2 + 1, 1, z2 + 2)

In [25]:
if c == c_decoded:
    mensaje = "<p style='color:green; font-size:16px;'></br>El resultado de la decodificación coincide con la codificación de la palabra original</p>"
else:
    mensaje = "<p style='color:red; font-size:16px;'></br>El resultado de la decodificación coincide no con la codificación de la palabra original</p>"

HTML(mensaje)

In [26]:
if message == message_decoded:
    mensaje = "<p style='color:green; font-size:16px;'></br>El mensaje decodificado coincide con el original</p>"
else:
    mensaje = "<p style='color:red; font-size:16px;'></br>El mensaje decodificado no coincide con el original</p>"

HTML(mensaje)

## Ejemplo 3

### Generación del código y la palabra a codificar

Creamos un código BCH sobre $\mathbb{F}_{3}$, de longitud $64$ y distancia diseñada $11$:

In [27]:
q = 3
F = GF(q)
n = 64
delta = 11
C = BCHCode(F, n, delta, None)
d = C.generator_matrix().nrows()

print("Código BCH:")
print(C)

print("\nMatriz de paridad:")
show(C.parity_check_matrix())

print("\nMatriz generadora:")
show(C.generator_matrix())

print("\nPolinomio generador:")
show(CyclicCode(code = C).generator_polynomial())

Código BCH:
[64, 10] BCH Code over GF(3) with designed distance 11

Matriz de paridad:


54 x 64 dense matrix over Finite Field of size 3 (use the '.str()' method to see the entries)


Matriz generadora:


10 x 64 dense matrix over Finite Field of size 3 (use the '.str()' method to see the entries)


Polinomio generador:


x^54 + 2*x^53 + x^52 + x^51 + x^49 + x^48 + x^38 + 2*x^37 + x^36 + x^35 + x^33 + x^32 + x^22 + 2*x^21 + x^20 + x^19 + x^17 + x^16 + x^6 + 2*x^5 + x^4 + x^3 + x + 1

Generamos una palabra aleatoria de tamaño $n$ de elementos del cuerpo finito, $\mathbb{F}_{5}$.

In [28]:
Encoder = BCHEncoder(C)

print("Codificador BCH:")
print(Encoder)

print("\nPalabra para codificar:")
message = random_word(d, F)
show(message)

Codificador BCH:
Encoder for [64, 10] BCH Code over GF(3) with designed distance 11

Palabra para codificar:


(0, 2, 2, 0, 2, 0, 0, 0, 0, 2)

Codificamos la palabra.

In [29]:
print("\nPalabra codificada:")
c = Encoder.encode(message)
show(c)


Palabra codificada:


(0, 2, 1, 2, 1, 0, 0, 2, 1, 0, 1, 0, 2, 2, 1, 2, 0, 2, 1, 2, 1, 0, 0, 2, 1, 0, 1, 0, 2, 2, 1, 2, 0, 2, 1, 2, 1, 0, 0, 2, 1, 0, 1, 0, 2, 2, 1, 2, 0, 2, 1, 2, 1, 0, 0, 2, 1, 0, 1, 0, 2, 2, 1, 2)

Generamos un vector de errores y se lo agregamos a la palabra codidicada.

In [30]:
print("\nVector de errores:")
n_errors = floor((delta-1)/2)
e = random_error(len(c), n_errors, F)
show(e)

print("\nPalabra codificada con errores (y = c + e):")
y = c + e
show(y)


Vector de errores:


(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)


Palabra codificada con errores (y = c + e):


(0, 2, 1, 2, 1, 0, 0, 2, 1, 0, 1, 0, 2, 2, 1, 2, 0, 2, 1, 2, 1, 0, 2, 2, 1, 0, 2, 0, 2, 2, 1, 2, 0, 2, 1, 2, 1, 0, 0, 2, 1, 0, 1, 0, 2, 2, 1, 2, 1, 2, 1, 2, 1, 0, 0, 2, 1, 0, 1, 0, 2, 2, 1, 2)

### Algoritmo de Peterson-Gorenstein-Zieler
Decodificamos con el algoritmo de Peterson-Gorenstein Zieler y comprobamos si coincide con el mensaje original.

In [31]:
Decoder = BCHDecoderPGZ(C)

print("\nDecodificador BCH con el algoritmo de Peterson-Gorenstein-Zieler:")
print(Decoder)

print("\nDecodificacion de y:")
c_decoded = Decoder.decode_to_code(y)
show(c_decoded)

print("\nMensaje decodificado de y:")
message_decoded = Decoder.decode_to_message(y)
show(message_decoded)


Decodificador BCH con el algoritmo de Peterson-Gorenstein-Zieler:
Peterson-Gorenstein-Zieler decoder for [64, 10] BCH Code over GF(3) with designed distance 11

Decodificacion de y:


(0, 2, 1, 2, 1, 0, 0, 2, 1, 0, 1, 0, 2, 2, 1, 2, 0, 2, 1, 2, 1, 0, 0, 2, 1, 0, 1, 0, 2, 2, 1, 2, 0, 2, 1, 2, 1, 0, 0, 2, 1, 0, 1, 0, 2, 2, 1, 2, 0, 2, 1, 2, 1, 0, 0, 2, 1, 0, 1, 0, 2, 2, 1, 2)


Mensaje decodificado de y:


(0, 2, 2, 0, 2, 0, 0, 0, 0, 2)

In [32]:
if c == c_decoded:
    mensaje = "<p style='color:green; font-size:16px;'></br>El resultado de la decodificación coincide con la codificación de la palabra original</p>"
else:
    mensaje = "<p style='color:red; font-size:16px;'></br>El resultado de la decodificación coincide no con la codificación de la palabra original</p>"

HTML(mensaje)

In [33]:
if message == message_decoded:
    mensaje = "<p style='color:green; font-size:16px;'></br>El mensaje decodificado coincide con el original</p>"
else:
    mensaje = "<p style='color:red; font-size:16px;'></br>El mensaje decodificado no coincide con el original</p>"

HTML(mensaje)

### Algoritmo de Sugiyama
Decodificamos con el algoritmo de Sugiyama y comprobamos si coincide con el mensaje original.

In [34]:
Decoder = BCHDecoderS(C)

print("\nDecodificador BCH con el algoritmo de Sugiyama:")
print(Decoder)

print("\nDecodificacion de y:")
c_decoded = Decoder.decode_to_code(y)
show(c_decoded)

print("\nMensaje decodificado de y:")
message_decoded = Decoder.decode_to_message(y)
show(message_decoded)


Decodificador BCH con el algoritmo de Sugiyama:
Sugiyama decoder for [64, 10] BCH Code over GF(3) with designed distance 11

Decodificacion de y:


(0, 2, 1, 2, 1, 0, 0, 2, 1, 0, 1, 0, 2, 2, 1, 2, 0, 2, 1, 2, 1, 0, 0, 2, 1, 0, 1, 0, 2, 2, 1, 2, 0, 2, 1, 2, 1, 0, 0, 2, 1, 0, 1, 0, 2, 2, 1, 2, 0, 2, 1, 2, 1, 0, 0, 2, 1, 0, 1, 0, 2, 2, 1, 2)


Mensaje decodificado de y:


(0, 2, 2, 0, 2, 0, 0, 0, 0, 2)

In [35]:
if c == c_decoded:
    mensaje = "<p style='color:green; font-size:16px;'></br>El resultado de la decodificación coincide con la codificación de la palabra original</p>"
else:
    mensaje = "<p style='color:red; font-size:16px;'></br>El resultado de la decodificación coincide no con la codificación de la palabra original</p>"

HTML(mensaje)

In [36]:
if message == message_decoded:
    mensaje = "<p style='color:green; font-size:16px;'></br>El mensaje decodificado coincide con el original</p>"
else:
    mensaje = "<p style='color:red; font-size:16px;'></br>El mensaje decodificado no coincide con el original</p>"

HTML(mensaje)